In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd

In [2]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(sentence):
  sentence = unicode_to_ascii(sentence.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)

  sentence = sentence.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  sentence = '<start> ' + sentence + ' <end>'
  return sentence

In [3]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
# def create_dataset(data):
#   # lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

#   data = [preprocess_sentence(sentence) for sentence in data]

#   return data 

In [4]:
df_train = pd.read_csv('/content/drive/My Drive/data/template/en-es-train-template.csv')
df_train=df_train.dropna(subset=['input_lang','output_lang'])


# df = pd.read_csv('/content/drive/My Drive/data/template/en-es-test-template.csv')
# df=df.dropna(subset=['input_lang','output_lang'])

# X_train, X_test, y_train, y_test = train_test_split(df["input_lang"], df["output_lang"], test_size=0.20, random_state=42)

In [5]:
df_en = [preprocess_sentence(text) for text in df_train["input_lang"].values]
df_es = [preprocess_sentence(text) for text in df_train["output_lang"].values]

# df_en = [preprocess_sentence(text) for text in X_train]
# df_es = [preprocess_sentence(text) for text in y_train]

In [6]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [7]:
def load_dataset(targ_lang, inp_lang):
  # creating cleaned input, output pairs
  # targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [8]:
# Try experimenting with the size of that dataset
# num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(df_es,df_en)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [9]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

228 228 58 58


In [10]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [11]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
2 ----> <start>
15 ----> what
22 ----> are
5 ----> the
345 ----> five
346 ----> boroughs
6 ----> of
1 ----> entity
4 ----> ?
3 ----> <end>

Target Language; index to word mapping
1 ----> <start>
4 ----> ¿
64 ----> cuales
35 ----> son
11 ----> los
487 ----> cinco
488 ----> condados
6 ----> de
489 ----> nueva
490 ----> york
3 ----> ?
2 ----> <end>


In [12]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [13]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 17]), TensorShape([64, 22]))

In [14]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [15]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 17, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [16]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [17]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 17, 1)


In [18]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [19]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 519)


In [20]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [21]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [22]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [23]:
EPOCHS = 100

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
  
  if total_loss<0.005:
    break

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.7709
Epoch 1 Loss 2.6980
Time taken for 1 epoch 45.40557098388672 sec

Epoch 2 Batch 0 Loss 2.4222
Epoch 2 Loss 2.3376
Time taken for 1 epoch 16.895276069641113 sec

Epoch 3 Batch 0 Loss 2.1680
Epoch 3 Loss 2.2177
Time taken for 1 epoch 16.598764419555664 sec

Epoch 4 Batch 0 Loss 1.9861
Epoch 4 Loss 2.0608
Time taken for 1 epoch 16.93299150466919 sec

Epoch 5 Batch 0 Loss 2.0625
Epoch 5 Loss 1.9669
Time taken for 1 epoch 16.551718711853027 sec

Epoch 6 Batch 0 Loss 1.8052
Epoch 6 Loss 1.8737
Time taken for 1 epoch 16.779998302459717 sec

Epoch 7 Batch 0 Loss 1.7569
Epoch 7 Loss 1.7624
Time taken for 1 epoch 16.68962812423706 sec

Epoch 8 Batch 0 Loss 1.8383
Epoch 8 Loss 1.7702
Time taken for 1 epoch 16.8481547832489 sec

Epoch 9 Batch 0 Loss 1.7052
Epoch 9 Loss 1.6878
Time taken for 1 epoch 16.47129511833191 sec

Epoch 10 Batch 0 Loss 1.6024
Epoch 10 Loss 1.6707
Time taken for 1 epoch 16.725659608840942 sec

Epoch 11 Batch 0 Loss 1.5986
Epoch 11 Loss 1.6063
Time

In [24]:
def evaluate(sentence):
  # attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]



  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    # attention_weights = tf.reshape(attention_weights, (-1, ))
    # attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      # return result, sentence, attention_plot
      return result, sentence
    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  # return result, sentence, attention_plot
  return result, sentence

In [25]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [26]:
def translate(sentence):
  # result, sentence, attention_plot = evaluate(sentence)
  result, sentence = evaluate(sentence)


  # print('Input: %s' % (sentence))
  # print('Predicted translation: {}'.format(result))
  # print(result)
  return result
  # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [27]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [28]:
# # df_test = pd.read_csv('/content/drive/My Drive/data/test/en-ro-test.csv')
df_test = pd.read_csv('/content/drive/My Drive/data/template/en-es-test-template.csv')
df_test=df_test.dropna(subset=['input_lang','output_lang'])
# # df_test = df_test[df_test.correctly_annotated==1]

In [29]:
# df_test.shape

In [30]:
count=0
w=0
e=0
for index, row in df_test.iterrows():
  try:
    trans = translate(row["input_lang"])
    sentence = preprocess_sentence(row["output_lang"])
    if trans[:-1]== sentence[8:]:
      count+=1
      # print(row["english"])
    else:
      w+=1
      print("expected: "+row["output_lang"]+"  "+"translation: "+trans) 
  except KeyError:
    e+=1
    print("error_sentence: " + row["input_lang"])
    continue

# df_en = [preprocess_sentence(text) for text in X_test]
# df_es = [preprocess_sentence(text) for text in y_test]

# for i in range(len(df_en)):
#   try:
#     trans = translate(df_en[i])
#     sentence = preprocess_sentence(df_es[i])
#     if trans[:-1]== sentence[8:]:
#       count+=1
#       # print(row["english"])
#     else:
#       w+=1
#       print("expected: "+df_es[i]+"  "+"translation: "+trans) 
#   except KeyError:
#     e+=1
#     print("error_sentence: " + df_en[i])
#     continue

expected: dame todos los actores de películas dirigidas por entidad en las qué también haya actúado el mismo.  translation: dame todas las que tambien haya actuado en las que tambien haya actuado en las que tambien haya actuado en las que 
expected: ¿qué altura tiene entidad?  translation: ¿ como de alta es entidad ? <end> 
expected: ¿cuál es el apodo de san francisco?  translation: ¿ cual es el hijo mas largo ? <end> 
expected: dame todos los astronautas de entidad?  translation: dame todas las taikonautas . <end> 
expected: ¿dónde vive el primer ministor de entidad?  translation: ¿ cuantas personas que estudio gobierna en el primer ministor de entidad ? <end> 
expected: dame las ciudades hermanadas con entidad.  translation: dame todos los nietos de entidad . <end> 
expected: ¿qué otras armas desarrolló el inventor del entidad?  translation: ¿ en que peliculas actuan entidad y entidad ? <end> 
expected: ¿en qué año fue fundada la entidad que produce entidad?  translation: ¿ en que an

In [31]:
count/df_test.shape[0]
# count/len(X_test)

0.38414634146341464

In [32]:
count

63

In [33]:
e/df_test.shape[0]

0.11585365853658537

In [34]:
w/df_test.shape[0]

0.5